In [25]:
import json
import requests
import sqlite3

In [26]:
coordinates = "55.7263,37.6503,55.7816,37.7864"
text_query = f""" [out: json];
    nwr({coordinates});
    out geom; """

In [27]:
overpass_url = "https://overpass.openstreetmap.ru/api/interpreter"
response = requests.get(overpass_url, params={'data': text_query})
json_data = response.json()
del json_data["version"]
del json_data["generator"]
del json_data["osm3s"]
with open("output_data.json", "w") as outfile:
    json.dump(json_data, outfile)

In [28]:
node_list = []
node_tags = []
tags_id = 1
for i in range(len(json_data["elements"])):
    if(json_data["elements"][i]["type"] == "node"):
        node_id = json_data["elements"][i]["id"]
        lat     = json_data["elements"][i]["lat"]
        lon     = json_data["elements"][i]["lon"]
        node_list.append((node_id, lat, lon))
        if('tags' in json_data['elements'][i].keys()):
            for k in json_data['elements'][i]['tags'].keys():
                tag_key = k
                tag_val = json_data['elements'][i]['tags'][k]
                node_tags.append((tags_id, node_id, tag_key, tag_val))
                tags_id += 1


In [57]:
way_list = []
way_tags = []
tags_id = 1
for i in range(len(json_data["elements"])):
    if(json_data["elements"][i]["type"] == "way"):
        way_id = json_data["elements"][i]["id"]
        way_list.append((way_id,))
        if('tags' in json_data['elements'][i].keys()):
            for k in json_data['elements'][i]['tags'].keys():
                tag_key = k
                tag_val = json_data['elements'][i]['tags'][k]
                way_tags.append((tags_id, node_id, tag_key, tag_val))
                tags_id += 1

In [58]:
with open('tests/way_tags_test.txt', 'w') as file:
    for x in way_tags:
        print(x, file=file)
file.close()

with open('tests/way_test.txt', 'w') as file:
    for x in way_list:
        print(x, file=file)
file.close()

In [64]:
con = sqlite3.connect('shadow.db')
cur = con.cursor()

#cur.executemany("INSERT INTO nodes VALUES (?, ?, ?)", node_list)
#cur.executemany("INSERT INTO node_tags VALUES (?, ?, ?, ?)", node_tags)
cur.executemany("INSERT INTO ways VALUES (?)", way_list)
#cur.executemany("INSERT INTO way_tags VALUES (?, ?, ?, ?)", way_tags)

con.commit()

In [65]:
cur.close()
con.close()